In [1]:
from pyspark import SparkContext, SparkConf
import time
conf = SparkConf().setAppName("fccRDD")
sc = SparkContext(conf=conf)
rdd = sc.parallelize([(0,1),(1,2),(1,3),(3,4),(5,6),(6,7),(7,8)])

23/04/17 01:34:13 WARN Utils: Your hostname, kimura.local resolves to a loopback address: 127.0.0.1; using 192.168.0.28 instead (on interface en0)
23/04/17 01:34:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/17 01:34:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
23/04/17 01:34:15 WARN Utils: Service 'SparkUI' could not bind on

In [2]:
def mapper(rdd):
    mapped = rdd.flatMap(lambda x: ((x[0],x[1]),(x[1],x[0])))
    mapped = mapped.groupByKey().mapValues(list).partitionBy(100).cache()
    return mapped.map(lambda x: (int(x[0]), list(map(int, x[1]))))

def reducer(edges):
    key, values = edges
    valueList, emit = [], []
    minim = min(values)
    valueList.extend(values)
    
    if  key <= minim:
        return [], 0
    
    emit.append((key, minim))
    for value in valueList:
        if minim != value:
            emit.append((value, minim))
    
    return emit, len(emit)-1

In [3]:
def find_ccp(rdd) :

    start, count, iterations = time.time(), 1, 0

    while count > 0:
        #CCFiterate
        mapp = mapper(rdd)
        reduce = mapp.map(lambda x: reducer(x))
        count = reduce.map(lambda x: x[1]).reduce(lambda x, y : x + y)
        result_reducer = reduce.flatMap(lambda x: x[0])
        
        #CCFdedup
        rdd = result_reducer.distinct()
        iterations+=1


    time_spent = time.time() - start
    
    print("time: ", time_spent)
    print("number of iterations:", iterations)
    print(rdd.collect())

In [4]:
find_ccp(rdd)

time:  12.763375282287598
number of iterations: 4
[(4, 0), (8, 5), (7, 5), (6, 5), (1, 0), (2, 0), (3, 0)]
